In [1]:
import numpy as np
import networkx as nx
import os

# Call this function to get the data 
def get_data(city='Detroit', mode='bike', root_path='../data/'):
    folders = os.listdir(root_path)
    for folder in folders:
        if folder == city:
            files = os.listdir(root_path+folder)
            bike_data = [file for file in files if mode in file][0]
            path = root_path + folder+ '/'+ bike_data;
    return nx.read_graphml(path)

G = get_data()
print(type(G))

<class 'networkx.classes.digraph.DiGraph'>


#### Define helper functions to compute metrics

In [2]:
def convert_lengths_to_floats(G):
    """
    Changes the attributes of the edges of graph G such that the 'length' field
    is a float rather than a string
    """
    for u, v, d in list(G.edges(data=True)):
        d['length'] = float(d['length'])

def get_edge_length(G, u, v):
    """
    given two nodes in graph G, return the straight line distance 
    (as the crow flies) between them
    """
    xdist = float(G.nodes[u]['x']) - float(G.nodes[v]['x'])
    ydist = float(G.nodes[u]['y']) - float(G.nodes[v]['y'])
    length = (xdist**2 + ydist**2)**0.5
    return length

def make_new_random_edge(G, u=None):
    """
    Return a tuple of nodes representing a random new and unique edge
    """
    if not u:
        u = np.random.choice(G.nodes(), 1)
        u = u[0]
    
    available = [node for node in G.nodes() if node not in G[u]]
    
    # choose v if not specified
    v = np.random.choice(available, 1)
    v = v[0]
    return (u, v)

    
def calc_directness(G, u, v):
    """
        Calculate the directness between two nodes u and v
        Directness is the ratio between:
            - the shortest node path length (number of edges times the length of each edge)
            - the straight line distance between those two nodes
        If v cannot be reached from u, then directness is zero
        
        G: An nx graph
        u: the starting node
        v: the ending node
        
    """
    try:
        path_length = nx.shortest_path_length(G, source=u, target=v, weight='length')
        direct_length = new_edge_length(G, u, v)
        ret = direct_length/path_length
    except:
        return 0

    return ret

def calc_avg_directness_connected(G, n):
    """
    Calculates the average directness of n pairs of nodes. All of these nodes are already connected.
    G: An nx graph
    n: The number of pairs to use
    """
    d = 0
    for _ in range(n):
        start = np.random.choice(G.nodes(), 1)
        start = start[0]
        reachable = nx.descendants(G, start)
        if not reachable:
            continue
        end = np.random.choice(list(reachable), 1)
        end = end[0]

        d += (calc_directness(G, start, end) / n);
    return d

def calc_avg_directness_random(G, n):
    """
    Calculates the average directness of n pairs of nodes. These nodes are chosen at random. 
    If they are not connected, the directness is zero
    
    G: An nx graph
    n: the number of pairs to use
    """
    d = 0
    for _ in range(n):
        pairs = np.random.choice(G.nodes(), 2)
        

def get_components(G):
    '''
    Get the connected components of G
    G: An nx graph
    
    Outputs
    wcc: A list of the components sorted by size
    '''
    wcc = [cc for cc in nx.weakly_connected_component_subgraphs(G)]
    wcc.sort(key=len, reverse=True)
    return wcc

def calc_lcc(G):
    '''
    Returns the number of nodes in the largest component of graph G
    G: An nx graph
    '''
    wcc = get_components(G)
    return len(wcc[0])

def euclidean_dist_vec(y1, x1, y2, x2):
    '''
    Calculate the euclidean distance between two points.
    '''
    distance = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
    return distance


#### Define Methods for connecting the Graphs

In [3]:
def L2S(wcc):
    '''
    Find the closest pair of nodes between two different connected components.
    ---
    wcc: list connected components

    returns: dict nodes i and j and distance
    '''
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    for i in wcc[0].nodes(data=True):
        i_coord = (i[1]['y'], i[1]['x'])
        for j in wcc[1].nodes(data=True):
            j_coord = (j[1]['y'], j[1]['x'])
            dist = euclidean_dist_vec(i_coord[0], i_coord[1], j_coord[0], j_coord[1])
            if dist < closest_pair['dist']:
                closest_pair['i'] = i[0]
                closest_pair['j'] = j[0]
                closest_pair['dist'] = dist
    return closest_pair

def L2C(wcc):
    pass

def R2C(wcc):
    pass

def Cloest(wcc):
    pass

In [5]:
convert_lengths_to_floats(G)
# Separate the graph into components
wcc = [cc for cc in nx.weakly_connected_component_subgraphs(G)]
wcc.sort(key=len, reverse=True)

ds = []
for component in wcc:
    num_edges = len(component.edges())
    ds.append(calc_avg_directness(component, int(num_edges**0.7)))
ds

[0.7648613051718686,
 0.7343356315721234,
 0.7420760143971832,
 0.8191443347954969,
 0.7378963701383855,
 1.001983400981681,
 0.999673581596141,
 0.8250864604793381,
 1.0015956574417908,
 0.9073006056780458,
 0.9379728621193912,
 0.971378981875917,
 0.9128755104835539,
 0.9786185919522088,
 0.9973638081128271,
 0.9127434890429086,
 1.0025561528889038,
 0.9853078566996968,
 0.9719435223624527,
 0.9120387125516936,
 0.999187971294352,
 0.9899101985054352,
 0.8957981162977794,
 0.9639994925681419,
 0.9401330550558847,
 1.0026145061358769,
 1.0018674212828291,
 0.9850177649429644,
 0.8780106608539596,
 1.0017630438157192,
 0.9993691531407217,
 1.0011685096977487,
 0.9966030709164675,
 0.9184484656604768,
 0.9991520825920828,
 1.0016509200733466,
 0.997665821974729,
 0.9989337809871051,
 1.001701033350583,
 0.9975729460346459,
 0.9989245374708997,
 0.999683877398664,
 1.0026345574253273,
 1.0017498600654813,
 1.001595830538935,
 1.001730718529688,
 0.9997617701457238,
 1.002588203849152,
 1

In [ ]:
def new_city(wcc, algy):
    """
    wcc = Sorted list of weakly connected subgraphs
    algy = desired path-adding algorithm
    
    """
    
    runs = len(wcc) - 1
    added_edges = []
    tot_length = []
    for i in range(runs):
        added_edge = algy(wcc[i:])
        edge = (added_edge['i'],added_edge['j'],added_edge['dist'])
        added_edges.append(edge)
        tot_length.append(edge[2])
    return added_edges, tot_length     